Run after first running the play by play data. Then you can obtain lineups using this code

In [ ]:
#lineups
zeros = ['0','0','0','0','0']
import numpy as np
pbp['lineup'] = pd.DataFrame(np.zeros((num,1)))
pbp['lineup'] = pbp['lineup'].astype(object)
for i in range(0,num):
    pbp.set_value(i, 'lineup', zeros)
    
#denoting the lineups in lists so I can sort at subs and compare...this alone took me a week...
lol = []
l=[]
lc = zeros[:]
t=0
for j,i in enumerate(alist):
    if pbp.ix[i,'desc'] not in ('In','Out'):
        if len(l) < 5:            
            if pbp.ix[i,'player'].split(',', 2)[0] not in l:
                l.extend([pbp.ix[i,'player'].split(',', 2)[0]])
                if len(l)==5:
                    l.sort()
                    lol.append(l)
        else:
            if pbp.ix[alist[j-1],'lineup'] == zeros:
                pbp.set_value(i,'lineup',lol[t])
            else:
                pbp.set_value(i,'lineup',pbp.ix[alist[j-1],'lineup'])
    elif pbp.ix[i,'desc'] == 'In':
        if len(l) < 5:
            l.extend([pbp.ix[i,'player'].split(',', 2)[0]])
            flag = 'sub'
        else:
            if pbp.ix[alist[j-1],'desc'] not in ('In','Out') and len(l)>4:
                lc = lol[t][:]
            lc.extend([pbp.ix[i,'player'].split(',', 2)[0]])
            lc.sort()
    elif pbp.ix[i,'desc'] == 'Out':
        if flag == 'sub':
            l.remove(pbp.ix[i,'player'].split(',', 2)[0])
            flag = 'good'
        else:
            lc.remove(pbp.ix[i,'player'].split(',', 2)[0])
            lc.sort()
            if i < max(alist):
                if pbp.ix[alist[j+1],'desc'] not in ('In','Out'):
                    lol.append(lc)
                    t+=1
                    
#copying down and up the lineups to unfilled cells
f=200
for i in range(0,num):
    if pbp.ix[i,'lineup'] <> zeros and i < f and pbp.ix[i,'team'] == away:
        f=i
        
for i in range(0,f):
    pbp.set_value(i,'lineup',pbp.ix[f,'lineup'])


for i in range(0,num):
    if pbp.ix[i,'lineup'] <> zeros:
        for u in range(i+1,num):
            if pbp.ix[u,'lineup'] <> zeros:
                break
            else:
                pbp.set_value(u,'lineup',pbp.ix[i,'lineup'])
                
#timer so that I can have criterium of over 300seconds
pbp['timer']=0
pbp.ix[0,'timer']=60-int(pbp.ix[0,'sec'])
for i in range(1,num):
    if int(pbp.ix[i-1,'sec'])>=int(pbp.ix[i,'sec']):
        pbp.ix[i,'timer']=int(pbp.ix[i-1,'sec'])-int(pbp.ix[i,'sec'])
    else:
        pbp.ix[i,'timer']=int(pbp.ix[i-1,'sec'])-int(pbp.ix[i,'sec'])+60
    
    
#spliting +/-
###Reversed points to reflect Oly away!!!!SOS!!!
pbp['hpts']=0
pbp['apts']=0
for i in range(0,num):
    if pbp.ix[i,'team']==home:
        pbp.ix[i,'apts']=pbp.ix[i,'pts']
        pbp.ix[i,'hpts']=0
    else:
        pbp.ix[i,'hpts']=pbp.ix[i,'pts']
        pbp.ix[i,'apts']=0
        
ptspbp=pbp.loc[:,['lineup','timer','hpts','apts']]
ptspbp.lineup = ptspbp.lineup.apply(tuple)
hf = ptspbp.groupby('lineup', as_index=False)['timer','hpts','apts'].sum()
hf = hf[hf.timer > 60]
hf['net']=hf['hpts']-hf['apts']
hf = hf.sort_values(by='net',ascending=False)
hf = hf.reset_index(drop=True)

###h1=hf.loc[:,:]

n=len(h1['lineup'])
num=len(hf['lineup'])
for i in range(0,num):
    for j in range(0,n):
        if h1.loc[j,'lineup'] == hf.loc[i,'lineup']:
            h1.loc[j,'timer'] = h1.loc[j,'timer'] + hf.loc[i,'timer']
            h1.loc[j,'hpts'] = h1.loc[j,'hpts'] + hf.loc[i,'hpts']
            h1.loc[j,'apts'] = h1.loc[j,'apts'] + hf.loc[i,'apts']
            h1.loc[j,'net'] = h1.loc[j,'hpts']-h1.loc[j,'apts']
            t=1
    if t==0:
        h1.loc[n]=hf.loc[i]
        n=len(h1['lineup'])
    else:
        t=0
        
best = h1[h1.timer > 600]
best = best.sort_values(by='net',ascending=False)
best = best.reset_index(drop=True)
best

Now we can plot the lineups

In [ ]:
import matplotlib.pyplot as plt
from __future__ import division

n=len(best['timer'])
ratio=[0]*25
for i in range(0,n):
    ratio[i]=100*best.loc[i,'timer']/72000
    
#n=len(best['timer'])
#w = pd.DataFrame(np.random.randn(n, 1))
#for i in range(0,n):
#    w.loc[i,0] = ((best.loc[i,'timer'] - min(best['timer'])) / (max(best['timer']) - min(best['timer'])))+0.5

x=range(0,25)
y=best['net']
fig, ax = plt.subplots(figsize = (20,10))
plt.bar(x, y, width=0.8, color=color_list)
plt.axis([-1, 26, -50, +50])
plt.plot([-2, 28], [0, 0], color='black', linestyle='-', linewidth=1)
ax.set_xticks([])
for i in range(1,n+1):
    ax.text(i-0.8, 1, i, fontsize=15)
for i in range(0,5):
    ax.text(13, 41-5*i,i+1 and '[%s]' % ', '.join(map(str, best.loc[i,'lineup'])) , style='italic')
for i in range(0,5):
    ax.text(12.4, 41-5*i-0.5, i+1, fontsize=15)
for i in range(20,25):
    ax.text(2, -i*5+85,i+1 and '[%s]' % ', '.join(map(str, best.loc[i,'lineup'])) , style='italic')
for i in range(20,25):
    ax.text(1.3, -i*5+85-0.5, i+1, fontsize=15)

color_list = ['#05AE00','#06C600','#6DC86A','#07DC00','#08F000','#0BFF02','#7CFF77',
              '#94FE5A','#B3FE5A','#CAFE5A','#D7FB8A','#DCFF00','#ECFF00','#F4FD84',
              '#FFCD00','#FF9E00','#FF6100','#FFAB78','#FB7726','#FF6100','#FF531C',
              '#FF1C1C','#FF5B5B','#FF3838','#FF0000']
ax.text(15, 46,'Most efficient lineups (+/-)', style='italic',
    bbox={'facecolor':'green', 'alpha':0.25, 'pad':10})
ax.text(21, 46,'% of total mins played', style='italic',
    bbox={'facecolor':'green', 'alpha':0.25, 'pad':10})
for i in range(0,5):
    ax.text(22, 40.5-5*i,(round(ratio[i],1)), style='italic')
    
ax.text(3, -10,'Least efficient lineups (+/-)', style='italic',
    bbox={'facecolor':'red', 'alpha':0.25, 'pad':10})
ax.text(10.5, -10,'% of total mins played', style='italic',
    bbox={'facecolor':'red', 'alpha':0.25, 'pad':10})
for i in range(20,25):
    ax.text(11, -5*i+84.5,(round(ratio[i],1)), style='italic')
    
plt.title('Lineup efficiency (restricted to lineups that played at least 10 minutes together)')
plt.ylabel('Net Rating (+/-)')
plt.xlabel('Lineup')
plt.show()